In [30]:
import MySQLdb
import pandas as pd

# Run the 3 commented lines below on the terminal
# ssh -f django@bcpp-clinic.bhp.org.bw -L5002:localhost:3306 -N
# ssh -f django@bhp066.bhp.org.bw -L5003:localhost:3306 -N
# ssh -f django@edc.bhp.org.bw -L5004:localhost:3306 -N
clinic = MySQLdb.connect(host='127.0.0.1', port=5002,user='root', passwd='cc3721b', db='edc')
bhp066 = MySQLdb.connect(host='127.0.0.1', port=5003,user='root', passwd='cc3721b', db='edc')
old_edc = MySQLdb.connect(host='127.0.0.1', port=5004,user='root', passwd='cc3721b', db='bhp066_master')

<html>
<h1>Generate Subject Requisitions from all Servers</h1>
    <ul>
        <li>Current Central Server</li>
        <li>Current Clinie Central Server</li>
        <li>Old Central Server</li>
    </ul>
</html>

<html>
<h3>Requitions Current Central Server</h3>
</html>

In [31]:
bhp066_req_sql_statement = 'SELECT sv.subject_identifier, sr.requisition_identifier, sr.drawn_datetime, sr.panel_name, sv.visit_code as year, p.map_area as community FROM bcpp_subject_subjectrequisition as sr LEFT JOIN bcpp_subject_subjectvisit as sv ON sr.subject_visit_id = sv.id LEFT JOIN member_householdmember as hm ON sv.household_member_id = hm.id LEFT JOIN household_householdstructure as hs ON hm.household_structure_id = hs.id LEFT JOIN household_household as h ON hs.household_id = h.id LEFT JOIN plot_plot as p ON h.plot_id = p.id'
bhp066_req_df = pd.read_sql(bhp066_req_sql_statement, con=bhp066)
bhp066_req_df.head()

,subject_identifier,requisition_identifier,drawn_datetime,panel_name,year,community
0,066-20380005-1,26AV2UU,2017-07-07 09:03:45,Research Blood Draw,T2,mmathethe
1,066-38190040-6,19WK4RM,2015-10-29 10:48:11,Microtube,T0,nata
2,066-351600001-0,16MAWP2,2017-11-07 10:25:11,Microtube,E0,gumare
3,066-30400032-4,356HFRHV,2016-12-10 09:15:01,Microtube,T1,tati_siding
4,066-26350009-6,41ZWFVM,2017-09-12 09:37:32,Microtube,T2,mmadinare


<html>
<h3>Requitions Current Clinic Central Server</h3>
</html>

In [32]:
clinic_req_sql_statement = 'SELECT sv.subject_identifier, sr.requisition_identifier, sr.drawn_datetime, sr.panel_name, sv.visit_code as year, se.map_area as community FROM bcpp_clinic_subject_subjectrequisition as sr LEFT JOIN bcpp_clinic_subject_subjectvisit as sv ON sr.subject_visit_id = sv.id LEFT JOIN bcpp_clinic_subject_subjectconsent as sc ON sv.subject_identifier = sc.subject_identifier LEFT JOIN bcpp_clinic_subject_subjecteligibility as se ON sc.screening_identifier = se.screening_identifier'
clinic_req_df = pd.read_sql(clinic_req_sql_statement, con=clinic)
clinic_req_df.head()

,subject_identifier,requisition_identifier,drawn_datetime,panel_name,year,community
0,066-21740015-0,746GCWN,2017-10-19 07:54:39,Viral Load,C0,lerala
1,066-31650006-6,65ARX8D,2017-09-18 09:02:06,Research Blood Draw,C0,mathangwane
2,066-31760005-5,76M8ZHK,2017-09-18 09:11:05,Viral Load,C0,mathangwane
3,066-18760003-7,76WW4EM,2018-01-22 06:06:32,Research Blood Draw,C0,oodi
4,066-18650013-9,65BPAAC,2018-02-19 08:26:31,Research Blood Draw,C0,oodi


<html>
<h3>Requitions Old Central Server</h3>
</html>

In [33]:
old_bhp066_req_sql_statement = 'SELECT sv.subject_identifier, sr.requisition_identifier, sr.drawn_datetime, p.name as panel_name, vd.code as year, pl.community FROM bcpp_lab_subjectrequisition as sr LEFT JOIN bcpp_lab_panel as p ON sr.panel_id = p.id LEFT JOIN bcpp_subject_subjectvisit as sv ON sr.subject_visit_id = sv.id LEFT JOIN bhp_appointment_appointment as ap ON sv.appointment_id = ap.id LEFT JOIN bhp_visit_visitdefinition as vd ON ap.visit_definition_id = vd.id LEFT JOIN bcpp_household_member_householdmember as hm ON sv.household_member_id = hm.id LEFT JOIN bcpp_household_householdstructure as hs ON hm.household_structure_id = hs.id LEFT JOIN bcpp_household_household as h ON hs.household_id = h.id LEFT JOIN bcpp_household_plot as pl ON h.plot_id = pl.id'
old_bhp066_req_df = pd.read_sql(old_bhp066_req_sql_statement, con=old_edc)
old_bhp066_req_df.head()

,subject_identifier,requisition_identifier,drawn_datetime,panel_name,year,community
0,066-38190040-6,19WK4RM,2015-10-29 10:48:11,Microtube,T0,nata
1,066-30400032-4,356HFRHV,2016-12-10 09:15:01,Microtube,T1,tati_siding
2,066-26220017-4,22E2CGT,2015-04-12 18:11:57,Microtube,T0,mmadinare
3,066-22270004-0,27A97RV,2015-02-13 15:29:38,Viral Load,T0,sefophe
4,066-13350016-0,356YEGD,2014-04-26 14:06:42,Microtube,T0,molapowabojang


In [34]:
requisition_df = bhp066_req_df.append(clinic_req_df)
requisition_df = requisition_df.append(old_bhp066_req_df)
print(len(requisition_df))
requisition_df.head()

84947


,subject_identifier,requisition_identifier,drawn_datetime,panel_name,year,community
0,066-20380005-1,26AV2UU,2017-07-07 09:03:45,Research Blood Draw,T2,mmathethe
1,066-38190040-6,19WK4RM,2015-10-29 10:48:11,Microtube,T0,nata
2,066-351600001-0,16MAWP2,2017-11-07 10:25:11,Microtube,E0,gumare
3,066-30400032-4,356HFRHV,2016-12-10 09:15:01,Microtube,T1,tati_siding
4,066-26350009-6,41ZWFVM,2017-09-12 09:37:32,Microtube,T2,mmadinare


In [35]:
requisition_df.to_csv('bcpp_all_servers_requisitions.csv', encoding='utf-8')

<html>
<h1>Generate Aliquotes from all Servers</h1>
<ul>
    <li>Current Central Server</li>
    <li>Current Clinie Central Server</li>
    <li>Old Central Server</li>
</ul>
</html>

<html>
<h3>Aliquots Current Central Server</h3>
</html>

In [36]:
# Central server connection
bhp066_aliquot_sql_statement = 'SELECT a.subject_identifier, a.aliquot_identifier, sr.requisition_identifier, sr.drawn_datetime, sr.panel_name, sv.visit_code as year, p.map_area as community FROM edc_lab_aliquot as a LEFT JOIN bcpp_subject_subjectrequisition as sr ON a.requisition_identifier = sr.requisition_identifier LEFT JOIN bcpp_subject_subjectvisit as sv ON sr.subject_visit_id = sv.id LEFT JOIN member_householdmember as hm ON sv.household_member_id = hm.id LEFT JOIN household_householdstructure as hs ON hm.household_structure_id = hs.id LEFT JOIN household_household as h ON hs.household_id = h.id LEFT JOIN plot_plot as p ON h.plot_id = p.id'
bhp066_aliquot_df = pd.read_sql(bhp066_aliquot_sql_statement, con=bhp066)
bhp066_aliquot_df_list = bhp066_aliquot_df.values.tolist()
for l in bhp066_aliquot_df_list:
    if not l[2]:
        print(l)
bhp066_aliquot_df.head()

['066-204400083-1', '06644DA77N02013601', None, NaT, None, None, None]
['066-357700003-2', '066779KYHT02013603', None, NaT, None, None, None]
['066-204400083-1', '06644DA77N02011201', None, NaT, None, None, None]
['066-351500661-2', '06615BB9GV02011205', None, NaT, None, None, None]
['066-351200116-0', '06612RGDXZ00000201', None, NaT, None, None, None]
['066-236200068-6', '0666283U8N02011203', None, NaT, None, None, None]
['066-351500661-2', '0661597UV202013602', None, NaT, None, None, None]
['066-351500661-2', '06615BB9GV02013604', None, NaT, None, None, None]
['066-205400081-2', '06654R8M6402013601', None, NaT, None, None, None]
['066-205400081-2', '06654DCGHF02013601', None, NaT, None, None, None]
['066-357700003-2', '066779KYHT02013602', None, NaT, None, None, None]
['066-351500661-2', '0661597UV202013603', None, NaT, None, None, None]
['066-205400081-2', '06654DCGHF02013602', None, NaT, None, None, None]
['066-357700004-0', '06677YGWG800000201', None, NaT, None, None, None]
['066-

,subject_identifier,aliquot_identifier,requisition_identifier,drawn_datetime,panel_name,year,community
0,066-241700051-7,06617XYVVV00000201,17XYVVV,2017-07-19 14:22:52,Microtube,E0,ramokgonami
1,066-235600023-9,0665693ECB02013601,5693ECB,2017-07-19 09:39:17,Microtube,E0,maunatlala
2,066-36380583-0,06655PGVN200000201,55PGVN2,2018-05-30 13:25:17,Research Blood Draw,T1,shakawe
3,066-351400040-0,06614YWAX300000201,14YWAX3,2017-11-21 13:55:59,Microtube,E0,gumare
4,066-40370007-5,06617X6VVX02013604,17X6VVX,2018-03-10 08:25:09,Viral Load,T2,tsetsebjwe


<html>
<h3>Aliquots Clinic Current Central Server</h3>
</html>

In [37]:
# Clinic Central server connection
clinic_aliquot_sql_statement = 'SELECT sv.subject_identifier, a.aliquot_identifier, sr.requisition_identifier, sr.drawn_datetime, sr.panel_name, sv.visit_code as year, se.map_area as community FROM edc_lab_aliquot as a LEFT JOIN bcpp_clinic_subject_subjectrequisition as sr ON a.requisition_identifier = sr.requisition_identifier LEFT JOIN bcpp_clinic_subject_subjectvisit as sv ON sr.subject_visit_id = sv.id LEFT JOIN bcpp_clinic_subject_subjectconsent as sc ON sv.subject_identifier = sc.subject_identifier LEFT JOIN bcpp_clinic_subject_subjecteligibility as se ON sc.screening_identifier = se.screening_identifier'
clinic_aliquot_df = pd.read_sql(clinic_aliquot_sql_statement, con=clinic)
clinic_aliquot_df_list = clinic_aliquot_df.values.tolist()
for l in clinic_aliquot_df_list:
    if not l[0]:
        print(l)
clinic_aliquot_df.head()

,subject_identifier,aliquot_identifier,requisition_identifier,drawn_datetime,panel_name,year,community
0,066-31710015-5,06671FNY8E02013204,71FNY8E,2017-10-25 06:47:29,Viral Load,C0,mathangwane
1,066-31760001-4,06676HU8EE02013206,76HU8EE,2017-09-06 08:17:01,Research Blood Draw,C0,mathangwane
2,066-21740013-5,066748M96M02013207,748M96M,2017-10-17 07:15:16,Viral Load,C0,lerala
3,066-25460004-3,06646CB7GR02013205,46CB7GR,2018-03-21 07:45:41,Research Blood Draw,C0,shoshong
4,066-31760013-9,06676XWMD202013204,76XWMD2,2017-10-05 07:38:12,Research Blood Draw,C0,mathangwane


<html>
<h3>Aliquots Old Central Server</h3>
</html>

In [ ]:
old_bhp066_aliquot_sql_statement = 'SELECT a.subject_identifier, a.aliquot_identifier, r.requisition_identifier, sr.drawn_datetime, p.name as panel_name, vd.code as year, pl.community FROM bcpp_lab_aliquot as a LEFT JOIN bcpp_lab_receive as r ON a.receive_identifier = r.receive_identifier LEFT JOIN bcpp_lab_subjectrequisition as sr ON r.requisition_identifier = sr.requisition_identifier LEFT JOIN bcpp_lab_panel as p ON sr.panel_id = p.id LEFT JOIN bcpp_subject_subjectvisit as sv ON sr.subject_visit_id = sv.id LEFT JOIN bhp_appointment_appointment as ap ON sv.appointment_id = ap.id LEFT JOIN bhp_visit_visitdefinition as vd ON ap.visit_definition_id = vd.id LEFT JOIN bcpp_household_member_householdmember as hm ON sv.household_member_id = hm.id LEFT JOIN bcpp_household_householdstructure as hs ON hm.household_structure_id = hs.id LEFT JOIN bcpp_household_household as h ON hs.household_id = h.id LEFT JOIN bcpp_household_plot as pl ON h.plot_id = pl.id'
old_bhp066_aliquot_df = pd.read_sql(old_bhp066_aliquot_sql_statement, con=old_edc)
old_bhp066_aliquot_df_list = old_bhp066_aliquot_df.values.tolist()
print(old_bhp066_aliquot_df_list[0])
for l in old_bhp066_aliquot_df_list:
    if not l[3]:
        print(l)
old_bhp066_aliquot_df.head()

In [ ]:
aliquot_df = bhp066_aliquot_df.append(clinic_aliquot_df)
aliquot_df = aliquot_df.append(old_bhp066_aliquot_df)
print(len(aliquot_df))
aliquot_df.head()

In [ ]:
aliquot_df.to_csv('bcpp_all_servers_aliquot.csv', encoding='utf-8')

In [ ]:
print("I am done go get your file.")